# 2유형

- 데이터 EDA -> 전처리 -> 모델링 -> 평가 -> 파일 저장
- 가이드 코드가 정해져 있음. 여러번 반복으로 플로우 암기하기
- 분류 또는 회귀 모델링 진행 후 예측 데이터 저장
    - 2회 : 분류 / roc_auc
    - 3회 : 분류 / roc_auc
    - 4회 : 분류 / f1-score
    - 5회 : 회귀 / RMSE
    - 6회 : 분류 / f1-score
    - 7회 : 회귀 / rmse
    - 8회 : 회귀 / mae
    ```python
    from sklearn.metrics import roc_auc_score, f1_score, mean_absolute_error, mean_squared_error
    
    # 모델에 대해 자체 평가를 할 때만 패키지 불러와서 확인
    # 시험에서는 예측 결과 파일만 제출하고 평가는 진흥원이 알아서함
    ```
- 모델 예측 결과를 파일 저장 (df.to_csv()), 파일 형식에 따라 감점요소 있음
    - 분류
        - 레이블 예측 (predict)
        - 확률 예측 (predict_proba)
        
    - 회귀
    
    ```python
    from sklearn.ensemble import RandomForestClassifier , RandomForestRegressor
    from xgboost import XGBRegressor, XGBClassifier
    
    rf = RandomForestClassifier()
    rf.fit(x_train)
    rf.predict(x_test) # 회귀, 분류 레이블
    rf.predict_proba(x_test)[:,1] # 확률 예측
    
    
    ```    
    
- 저장된 파일만으로 평가 (10점 단위)
    ```python
    predict = rf.predict(x_test)
    predict_df = pd.DataFrame(predict,columns =['pred'])
    predict_df.to_csv('result.csv',index=False)
    
    # 저장 후 데이터 불러와서 데이터 타입 print 해보기
    
    ```    
    


# 데이터 EDA

- 데이터 기본 정보 및 특이사항 파악
- shape,describe , info, unique, nunique, isnull().sum(), select_dtypes

In [1]:
train_url = 'https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/main/data/customer_train.csv'
test_url  = 'https://raw.githubusercontent.com/Datamanim/datarepo/refs/heads/main/data/customer_test.csv'

import pandas as pd

train = pd.read_csv(train_url)
test = pd.read_csv(test_url)

# 전처리

- 불필요 컬럼 제거
- 결측치처리
- 인코딩
- 스케일링
- 더미화
- 독립변수, 종속변수 분리
- 학습, 훈련 데이터 분할
    - train, test

In [2]:
from sklearn.preprocessing import LabelEncoder
remove_col = ['회원ID', '환불금액']

train_preprocessing = train.copy()
train_preprocessing = train_preprocessing.drop(columns = remove_col)
test_preprocessing = test.copy()

le = LabelEncoder()
le.fit(train_preprocessing['주구매상품'])
le_predict = le.fit_transform(train_preprocessing['주구매상품'])

le2 = LabelEncoder()
le2.fit(train_preprocessing['주구매지점'])
le2_predict = le2.fit_transform(train_preprocessing['주구매지점'])

train_preprocessing['주구매상품'] = le_predict
train_preprocessing['주구매지점'] = le2_predict

from sklearn.preprocessing import StandardScaler

scaled_df = train_preprocessing[['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']]
sc = StandardScaler()
sc.fit(scaled_df)
transform_data = sc.fit_transform(scaled_df)
train_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data

y = train_preprocessing['성별']
x = train_preprocessing.drop(columns= ['성별'])

from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(x, y, train_size=0.7, random_state=42)


C:\Users\Admin\AppData\Local\Temp\ipykernel_15812\1636056969.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.14458009 -0.54918957 -0.5426996  ... -0.56179637 -0.55078606
  1.04709431]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data
C:\Users\Admin\AppData\Local\Temp\ipykernel_15812\1636056969.py:25: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.26260786 -0.54796686 -0.56350404 ... -0.61239772 -0.58348042
  0.46792117]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  train_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data
C:\Users\Admin\AppData\Local\Temp\ipykernel_15812\1636056969.py:25: FutureWarning: Setting an item of incompatib

In [3]:
y_val

1650    1
2456    1
2232    1
1945    0
309     0
       ..
2670    0
1253    0
801     1
2112    0
1374    0
Name: 성별, Length: 1050, dtype: int64

# 모델링
- 회귀, 분류 확인하여 모델 선언
- 학습데이터로 모델 학습
    - (option) 그리드 서치로 최적 파라미터 선정 / 안해도 무관
    - 랜포가 짱
- 평가데이터에 대해 모델 예측

In [4]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(random_state=1)
rf.fit(X_train, y_train) # 학습
val_predict = rf.predict(X_val) # 예측
val_predict_train = rf.predict(X_train) # 예측

In [5]:
from xgboost import XGBClassifier
xgb = XGBClassifier()
xgb.fit(X_train, y_train)
val_xgb_predict = xgb.predict(X_val)

# 평가

In [6]:
from sklearn.metrics import accuracy_score
accuracy_score(y_val, val_predict)
accuracy_score(y_val, val_xgb_predict)

0.6066666666666667

In [7]:
from sklearn.metrics import roc_auc_score
val_rf_proba = rf.predict_proba(X_val)
roc_auc_score(y_val, val_rf_proba[:,1])
roc_auc_score(y_val, val_xgb_predict)

0.5613544939424087

In [10]:
from sklearn.metrics import f1_score
f1_score(y_train, val_predict_train)
f1_score(y_val, val_predict)

0.3860759493670886

In [13]:
# test 테스트 데이터에 대해서도 평가를함
# test 데이터를 train 데이터의 전처리과정과 동일하게 전처리 해줌
from sklearn.preprocessing import LabelEncoder
remove_col = ['회원ID', '환불금액']

test_preprocessing = test.copy()
test_preprocessing = test_preprocessing.drop(columns=remove_col)

le = LabelEncoder()
le.fit(test_preprocessing['주구매상품'])
le_predict_test = le.fit_transform(test_preprocessing['주구매상품'])

le2 = LabelEncoder()
le2.fit(test_preprocessing['주구매지점'])
le2_predict_test = le2.fit_transform(test_preprocessing['주구매지점'])

test_preprocessing['주구매상품'] = le_predict_test
test_preprocessing['주구매지점'] = le2_predict_test

from sklearn.preprocessing import StandardScaler

scaled_df_test = test_preprocessing[['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']]
sc = StandardScaler()
sc.fit(scaled_df_test)
transform_data_test = sc.fit_transform(scaled_df_test)
test_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data_test


C:\Users\Admin\AppData\Local\Temp\ipykernel_15812\1396642587.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[-0.1739659   1.20976788  1.17934283 ...  0.91799171 -0.06950064
 -0.57977131]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data_test
C:\Users\Admin\AppData\Local\Temp\ipykernel_15812\1396642587.py:26: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.0065498   0.47906677 -0.19820375 ...  0.11356385 -0.10417407
 -0.6153017 ]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  test_preprocessing.loc[:,['총구매액', '최대구매액', '방문일수', '방문당구매건수', '구매주기']] = transform_data_test
C:\Users\Admin\AppData\Local\Temp\ipykernel_15812\1396642587.py:26: FutureWarning: Setting an item of in

In [14]:
rf_test_predict = rf.predict(test_preprocessing)
xgb_test_predict = xgb.predict(test_preprocessing)

In [16]:
rf_test_predict_proba = rf.predict_proba(test_preprocessing)[:,1]

# 파일저장

In [ ]:
predict_df = pd.DataFrame(rf_test_predict, columns=['pred'])
predict_df.to_csv('result.csv', index=False)